In [48]:
import os
import re
import shutil
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser
import src.Parser.bench_parser as bench_parser
import src.Parser.conv as conv
from src.Netlist.AST import AST,module
import src.Attacks.SATAttack.SATAttack as satattack
import src.Attacks.SATAttack.benchmarks_custom as benchmarks

In [ ]:
# obj=AST(file_path=unlocked_filename,rw="w",flag="b",top=top,filename=f"{top}org",output_dir_path="output_files/ASSURE_BENCH",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design

In [5]:
# top="test_adder"
# netlist = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

In [6]:
# locked_filename="input_files/benchmark_bench/rnd/apex2_enc05.bench"
# unlocked_filename="input_files/benchmark_bench/original/apex2.bench"



top="sasc_top_0_obf"
locked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format
top="sasc_top"
unlocked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format



AttributeError: module 'os' has no attribute 'isdir'

In [ ]:
from node import Node
import attacks.satattack.circuit as circuit
import src.Attacks.SATAttack.benchmarks_custom as benchmarks

class cone_ckt:
  def __init__(self,outputs,primary_inputs,key_inputs,nodes,depth) -> None:
    self.primary_inputs=primary_inputs
    self.key_inputs=key_inputs
    self.nodes=nodes
    self.outputs=outputs
    self.depth=depth

class SATobj:
  def __init__(self,oracle_ckt:cone_ckt,locked_ckt:cone_ckt) -> None:
    self.oracle_ckt_cone=oracle_ckt
    self.locked_ckt_cone=locked_ckt
    self.keyinputs=self.locked_ckt_cone.key_inputs
    self.prim_inputs=self.locked_ckt_cone.primary_inputs
    
    self.oracle_ckt=circuit.Circuit.from_nodes(self.oracle_ckt_cone.nodes, self.oracle_ckt_cone.outputs)
    self.locked_ckt=(self.locked_ckt_cone.nodes, self.locked_ckt_cone.outputs)


class Analyze_Netlist:
  def __init__(self,locked_module_obj=None,unlocked_module_obj=None) -> None:
    self.locked=locked_module_obj
    self.unlocked=unlocked_module_obj
    self.attack_outputs={}


    if(self.locked!=None and self.unlocked!=None):
      self.common_outputs=utils.get_common_elements(self.locked.io["outputs"],self.unlocked.io["outputs"])
      self.diff_outputs=utils.get_difference(self.locked.io["outputs"],self.unlocked.io["inputs"])
      self.gen_cone_data()

    
  def gen_cone_ckts(self):
    pass

  def gen_cone_data(self):

    def traverse_graph_r(obj,out_node):
      if(obj.circuitgraph.nodes[out_node]["type"]=="input"):
          if("key" in out_node and out_node not in self.keyinputs):
            self.keyinputs.append(out_node)
            self.nodes[out_node]=Node(out_node,[],"Key Input")
          elif(out_node not in self.primary_inp):
              self.primary_inp.append(out_node)
              self.nodes[out_node]=Node(out_node,[],"Primary Input")
          return 
      elif(obj.circuitgraph.nodes[out_node]["type"]=="output"):
          if(out_node not in self.out):
            self.out.append(out_node)
      elif(out_node in self.nodes):
        return
      
      pred=obj.circuitgraph.predecessors(out_node)
      self.depth+=1
      for i in pred:
        if(obj.circuitgraph.nodes[i]["type"]=="gate"): 
          self.nodes[out_node]=Node(out_node,list(obj.circuitgraph.predecessors(i)),utils.det_logic(obj.circuitgraph.nodes[i]["logic"],obj.gates))
          # print("THERE ",i,out_node,list(obj.circuitgraph.predecessors(out_node)))
        # else:
        #   print("THERE ",i,out_node,list(obj.circuitgraph.predecessors(out_node)))
        
        # if("G180gat" in out_node):
            
        traverse_graph_r(obj,i) 
    
    
    
    self.common_outputs.sort(key=lambda s: int(''.join(filter(str.isdigit,s)) or 0))
    for i in self.common_outputs:
      print(i)
      pred1=list(self.unlocked.circuitgraph.predecessors(i))
      pred2=list(self.locked.circuitgraph.predecessors(i))
      if not pred2 or pred1:
        print(pred2,pred1)
        continue

      self.primary_inp=[]
      self.keyinputs=[]
      self.out=[]
      self.nodes={}
      self.depth=0
      traverse_graph_r(self.unlocked,i)
      if(i not in self.out):
        self.out.append(i)
      self.out.sort(key=lambda s: int(''.join(filter(str.isdigit,s)) or 0))
      oracle_ckt=cone_ckt(self.out,self.primary_inp,self.keyinputs,self.nodes,self.depth)
      

      # print(i,out,len(keyinputs),len(primary_inp),len(nodes),self.depth)

      self.primary_inp=[]
      self.keyinputs=[]
      self.out=[]
      self.nodes={}
      self.depth=0
      traverse_graph_r(self.locked,i)
      if(i not in self.out):
        self.out.append(i)
      self.out.sort(key=lambda s: int(''.join(filter(str.isdigit,s)) or 0))
      locked_ckt=cone_ckt(self.out,self.primary_inp,self.keyinputs,self.nodes,self.depth)
      # print('po2' in nodes,'po1' in nodes,'po0' in nodes)
      # print(i,out,len(keyinputs),len(primary_inp),len(nodes),self.depth,"\n")

      # try:
      # print(nodes)
      self.attack_outputs[i]=SATobj(oracle_ckt,locked_ckt)
      
      if(oracle_ckt.outputs==locked_ckt.outputs and locked_ckt.outputs==self.common_outputs):
        break
      # except KeyError as err:
      #   print(err)


 

    

  
  # def update_ckts(self):
  #   self.oracle_ckt=circuit.Circuit.from_nodes(self.nodes_org, self.attack_outputs)
  #   self.locked_ckt=(self.nodes_lock, self.attack_outputs)

In [ ]:
analyze=Analyze_Netlist(locked.top_module,unlocked.top_module)

In [ ]:
for i,j in zip(locked.top_module.io['outputs'],unlocked.top_module.io['outputs']):
    print(i,j)
    pred1=list(unlocked.top_module.circuitgraph.predecessors(j))
    pred2=list(locked.top_module.circuitgraph.predecessors(i))
    print(pred1,pred2)

In [42]:
analyze.attack_outputs

{}

In [66]:
for ii in analyze.attack_outputs:
  print("START SAT ATTACK")
  satobj=analyze.attack_outputs[ii]
  satattackobj=satattack.SatAttack(file_type="obj",satobj=satobj)
  # satobj=satattack.SatAttack(file_type="b",locked_filename=locked_filename, unlocked_filename=unlocked_filename)
  print(satattackobj.run())
  print(satattackobj.iterations)
  # break

START SAT ATTACK
(['keyinput1', 'keyinput0'], '10')
2
START SAT ATTACK
(['keyinput5', 'keyinput4', 'keyinput3', 'keyinput2', 'keyinput1', 'keyinput0'], '010110')
22
START SAT ATTACK
(['keyinput6', 'keyinput5', 'keyinput4', 'keyinput3', 'keyinput2', 'keyinput1', 'keyinput0'], '0010110')
7
START SAT ATTACK
(['keyinput7', 'keyinput6', 'keyinput5', 'keyinput4', 'keyinput3', 'keyinput2', 'keyinput1', 'keyinput0'], '00010110')
12
START SAT ATTACK
(['keyinput7', 'keyinput6', 'keyinput5', 'keyinput4', 'keyinput3', 'keyinput2', 'keyinput1', 'keyinput0'], '00010110')
6
START SAT ATTACK
(['keyinput7', 'keyinput6', 'keyinput5', 'keyinput4', 'keyinput3', 'keyinput2', 'keyinput1', 'keyinput0'], '00010110')
10
START SAT ATTACK
(['keyinput7', 'keyinput6', 'keyinput5', 'keyinput4', 'keyinput3', 'keyinput2', 'keyinput1', 'keyinput0'], '00010110')
9


In [26]:
locked.top_module.io["outputs"],unlocked.top_module.io["outputs"]

({'po2': {'bits': 1}, 'po1$enc': {'bits': 1}, 'po0': {'bits': 1}},
 {'po2': {'bits': 1}, 'po1': {'bits': 1}, 'po0': {'bits': 1}})

In [ ]:
satattackobj.

In [ ]:
satattackobj._key_string(satattackobj.key)

In [ ]:
satattackobj.key

In [ ]:
# import pickle
# pickle.dump(std1, file)

In [ ]:
top="fsm_0_obf"
pathin="input_files/ASSURE_LOCKED/design1/design1_netlist.v"

# top="fsm"
# pathin="input_files/ASSURE_LOCKED/design1/oracle1_netlist.v"

# utils.clean_dir("./tmp")

# obj=AST(file_path=pathin,rw="w",flag="v",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design

locked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

In [ ]:
# gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [ ]:
# locked.top_module.gates.keys(),locked.top_module.FF_tech.keys()

In [ ]:
# tech_map={i:i for i in list(locked.top_module.gates,locked.top_module.FF_tech)}

In [ ]:
# for gate, gate_list in gate_mapping_vlib.items():
#   print(gate, gate_list)

In [73]:
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
# path= "vlib/mycells.v"
# path="input_files/warm-up_obfuscation/internal_lib.v"
path="/home/alira/FYP_FINAL/input_files/ASSURE_LOCKED/modulefiles.v"
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
with open(path,"r") as f:
  module_txt=f.read()
gate_mapping_vlib2,gates_vlib2,FF_vlib2=verilog_parser.extract_modules_def(module_txt)

extract_modules_def took 0.0022084712982177734 seconds to complete its execution.


In [ ]:
gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [13]:
tech_map=gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2)

In [ ]:
txt=gates_to_txt_techmap(locked.top_module.gates,gates_vlib2,tech_map)
txt+=FF_to_txt_techmap(locked.top_module.FF_tech,FF_vlib2,tech_map)

In [22]:
with open("tmp.v","w") as f:
    f.write(txt)

In [ ]:
# path_dir=r"/mnt/d/alis_files/LAPTOP/alis_files/university_files/GDRIVE/Hardware security FYP/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules"
# paths=[os.path.join(path_dir,i) for i in os.listdir(path_dir) if(i[:5]=="Train")]

# import re
# def analyze_init(paths):
#   items={}
#   for path in paths:
#     with open(path,"r") as f:
#       module_txt=f.read()
    
#     for i in re.findall(r"(\w+) (\w+) \((.*)\);",module_txt)[1:]:
#       tmpi=re.findall(r'\.(\S+)\( ?[^\(\),]+\)',i[2])
#       port="("
#       for k in tmpi[:-1]:
#         port+=f".{k}({{{k}}}), "
#       port+=f".{tmpi[-1]}({{{tmpi[-1]}}})"+");"
#       if(i[0] not in items):
#         items[i[0]]=[port]
#       elif(items[i[0]]!=[port]):
#         raise Exception("Items have mismatching initiation")
#   return items

In [ ]:
# items=analyze_init(paths)

In [ ]:

# with open("input_files/ASSURE_LOCKED/modulefiles.v","r") as f:
#   module_txt=f.read()


# with open("tmp/tmpmod.v","w") as f:
#   f.write(module_txt)


# # with open("tmp.v","w") as f:
# #   f.write(t)

# gate_mapping,gates,FF=verilog_parser.extract_modules_def(module_txt)

# path="input_files/ASSURE_LOCKED/design4/design4_netlist.v"
# # "input_files/test.v"
# # "input_files/ASSURE_LOCKED/design4/design4_netlist.v"




# # # verilog=tmp1()
# # verilog=tmp2()
# # with open("tmp/tmp.v","w") as f:
# #   f.write(verilog)
# # # print(verilog)

# # print("HERE")


In [ ]:
# gate_tech,sub_module,(FF_tech,Clock_pins,Reset_pins)=verilog_parser.gates_module_extraction(verilog,gate_mapping,gates,FF)

In [ ]:
# json_files=["./output_files/"+i for i in os.listdir("./output_files/") if os.path.isfile("./output_files/"+i)]
# orgfile_names=[i.split(".bench")[0] for i in os.listdir("input_files/benchmark_bench/original")]
# json_files.sort()

# return_top =lambda i:re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(open(i).read()), re.DOTALL)[0][1]

# orgfile_names=[[return_top(f"input_files/ASSURE_LOCKED/{i}/{k}") for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if("oracle" in k and ".v" in k)][0] for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]

# for i in json_files:
#   for j in orgfile_names:
#     if(j in i.split("/")[-1].split("org")[0]):
#       print(i,"output_files/ASSURE_VERILOG/"+i.split("/")[-1])
#       shutil.move(i,"output_files/ASSURE_VERILOG/"+i.split("/")[-1])

In [10]:
# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()

In [ ]:
# import re
# import os
# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()
# notdone=[]
# for i in files[1:]:
#   design=i[0]
#   oracle=i[1]

#   tmp=open(design).read()
#   top_design = re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(tmp), re.DOTALL)[0][1]
#   tmp=open(oracle).read()
#   top_oracle = re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(tmp), re.DOTALL)[0][1]
#   # try:
#   #   print("DONE ",top_oracle)
#   obj=AST(file_path=oracle,rw="w",flag="v",top=top_oracle,filename=f"{top_oracle}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   # except:
#   #   print("FAILED ",top_oracle)
#   #   notdone.append(oracle)
#   # try:
#   print("DONE ",top_design)
#   obj=AST(file_path=design,rw="w",flag="v",top=top_design,filename=f"{top_design}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#   # except:
#   #   print("FAILED ",top_design)
#   #   notdone.append(design)
#   break

In [ ]:
# notdone=[]
# for i in os.listdir("input_files/benchmark_bench/original"):
#   top=i.split(".bench")[0]
#   unlocked_filename=os.path.join("input_files/benchmark_bench/original",i)
#   try:
#     obj=AST(file_path=unlocked_filename,rw="w",flag="b",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   except:
#     notdone.append(unlocked_filename)
#   for j in os.listdir("input_files/benchmark_bench/rnd"):
#     if(i.split(".bench")[0] in j):
#       top=j.split(".bench")[0]
#       locked_filename=os.path.join("input_files/benchmark_bench/rnd",j)
#       print(j.split(".bench")[0],locked_filename)
#       try:
#         obj=AST(file_path=locked_filename,rw="w",flag="b",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#       except:
#         notdone.append(locked_filename)